### author by yangshichen
### 注意：脚本仅供参考，使用前请仔细阅读

In [1]:
import os
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import seaborn as sb
import seaborn as sns
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt
from scipy.io import mmread
from scipy.sparse import csr_matrix
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from scipy.sparse import issparse
import decoupler as dc
from sklearn.preprocessing import QuantileTransformer
from multiprocessing import Pool, cpu_count
from functools import partial
from sklearn.preprocessing import QuantileTransformer, StandardScaler
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
scaler = StandardScaler()

import warnings
warnings.filterwarnings("ignore")

In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, frameon=False)
sc._settings.ScanpyConfig.n_jobs=70

scanpy==1.9.8 anndata==0.9.2 umap==0.5.7 numpy==1.24.4 scipy==1.10.1 pandas==2.0.3 scikit-learn==1.3.2 statsmodels==0.14.1 pynndescent==0.5.13


In [3]:
import os
os.environ["R_HOME"] = "/home/yangshichen//mambaforge/envs/QTL/lib/R"
os.environ["R_LIBS_USER"] = "/home/yangshichen/mambaforge/envs/QTL/lib/R/library"
import pandas as pd
import torch
import tensorqtl
from tensorqtl import  cis
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"torch: {torch.__version__} (CUDA {torch.version.cuda}), device: {device}")
print(f"pandas: {pd.__version__}")

torch: 2.4.1+cu121 (CUDA 12.1), device: cuda
pandas: 2.0.3


### scRNA-seq数据处理

#### 构建伪bulk数据

In [4]:
adata=sc.read("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/HIV-pbmc/pbmc_celltype.h5ad")
adata

AnnData object with n_obs × n_vars = 2744009 × 21679
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'doublet', 'batch', 'sample', 'stage', 'experiments', 'age', 'total_counts_rp', 'pct_counts_rp', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_LOC', 'pct_counts_LOC', 'total_counts_erccs', 'pct_counts_erccs', 'celltype_L1', 'phase_ordered', 'celltype_L3', 'celltype_L1_5', 'celltype_L2'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mean', 'std'
    uns: 'age_colors', 'celltype_L2_colors', 'experiments_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'phase_ordered_colors', 'stage_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [8]:
# 去除（X、Y、MT）染色体上的基因
annotation_df = pd.read_csv("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/gene_annotation.txt", sep='\t')
annotation_df = annotation_df[~annotation_df['chr'].isin(['chrX', 'chrY', 'chrMT'])].copy()
annotation_df

,gene_id,chr,left,right
0,DDX11L17,chr1,182696,184174
1,PRDM16,chr1,3069168,3438621
2,PEX10,chr1,2403964,2413797
3,RPL21P21,chr1,10054445,10054781
4,LINC01345,chr1,3944547,3949024
...,...,...,...,...
38660,ACTG1P14,chr9,6834456,6835577
38661,RPL4P5,chr9,7477045,7478320
38662,PPIAP33,chr9,7597823,7598377
38663,DMAC1,chr9,7796500,7888380


In [11]:
adata = adata[:, adata.var_names.isin(annotation_df['gene_id'])]
adata

View of AnnData object with n_obs × n_vars = 2744009 × 20078
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'doublet', 'batch', 'sample', 'stage', 'experiments', 'age', 'total_counts_rp', 'pct_counts_rp', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_LOC', 'pct_counts_LOC', 'total_counts_erccs', 'pct_counts_erccs', 'celltype_L1', 'phase_ordered', 'celltype_L3', 'celltype_L1_5', 'celltype_L2'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mean', 'std'
    uns: 'age_colors', 'celltype_L2_colors', 'experiments_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'phase_ordered_colors', 'stage_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [12]:
# 获取所有 celltype_L3 的唯一值
celltypes = adata.obs['celltype_L3'].unique()
celltypes

['Switched_Memory_B-CD27', 'Unswitched_Memory_B-CD1C', 'Switched_Memory_B-CD86', 'Naive_B-TCL1A', 'Switched_Memory_B-IGHE', ..., 'NK_bright-XCL1', 'Mature_NK_dim-FCGR3A', 'Terminal_NK_dim-CD160neg', 'Transitional_NK-GZMK', 'Inflamed_NK_dim-IFI44L']
Length: 58
Categories (58, object): ['Aptypical_Memory_B-ITGAX', 'CD4_Naive_T-CCR7', 'CD4_Naive_T-SOX4', 'CD4_Tcm-CXCR5', ..., 'ncMono-IFI44L', 'ncMono-IFIT1', 'pDC-AXL', 'pDC-LILRA4']

In [13]:
for celltype in celltypes:
    # 提取该细胞类型的子集
    adata_sub = adata[adata.obs['celltype_L3'] == celltype].copy()
    adata_sub.X = adata_sub.layers['counts']

    #算伪Bulk
    pdata = dc.get_pseudobulk(
            adata_sub,
            sample_col='sample',
            groups_col=None,
            mode='mean',
            min_cells=10,
            min_counts=0,
            min_prop=0,
            min_smpls=0)
        
    #生成伪bulk矩阵
    pseudo_matrix = pd.DataFrame(pdata.X)
    pseudo_matrix.columns = pdata.var_names
    pseudo_matrix.index = pdata.obs.index

    #只选取在90%的样本中都表达的特征
    non_zero_ratio = (pseudo_matrix != 0).mean()
    columns_to_keep = non_zero_ratio[non_zero_ratio >= 0.9].index
    pseudo_matrix = pseudo_matrix[columns_to_keep]

    # 计算每列的均值和标准差
    means = pseudo_matrix.mean()
    stds = pseudo_matrix.std()
    # 计算变异系数
    cv = (stds / means).abs() * 100
    # 按变异系数从大到小排序
    sorted_columns = cv.sort_values(ascending=False).index
    # 选取变异系数最高的前2000列
    top_2000_columns = sorted_columns[:min(2000,pseudo_matrix.shape[1])]

    #保存数据
    if pseudo_matrix.shape[1] > 0:
        normalized_pseudo_matrix = quantile_transformer.fit_transform(pseudo_matrix)
        normalized_pseudo_matrix = scaler.fit_transform(normalized_pseudo_matrix)
        normalized_pseudo_matrix = pd.DataFrame(normalized_pseudo_matrix, columns=pseudo_matrix.columns,index=pseudo_matrix.index)
        normalized_pseudo_matrix.to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/02.normal_dis/{celltype}.csv')
        normalized_pseudo_matrix[top_2000_columns].to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/04.top2000_normal_dis/{celltype}.csv')
        pseudo_matrix.to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/01.pseudobulk/{celltype}.csv')
        pseudo_matrix[top_2000_columns].to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/03.top2000_pseudobulk/{celltype}.csv')
    print(f"处理完成 {celltype} (genes retained: {len(columns_to_keep)})", flush=True)

处理完成 Switched_Memory_B-CD27 (genes retained: 7653)
处理完成 Unswitched_Memory_B-CD1C (genes retained: 8927)
处理完成 Switched_Memory_B-CD86 (genes retained: 7932)
处理完成 Naive_B-TCL1A (genes retained: 10344)
处理完成 Switched_Memory_B-IGHE (genes retained: 6490)
处理完成 Aptypical_Memory_B-ITGAX (genes retained: 7100)
处理完成 Transitional_B-NEIL1 (genes retained: 6548)
处理完成 Naive_B-IFIT3 (genes retained: 2672)
处理完成 Naive_B-IL6 (genes retained: 1575)
处理完成 Unswitched_Memory_B-IFIT3 (genes retained: 2887)
处理完成 Unswitched_Memory_B-JAM3 (genes retained: 6802)
处理完成 Plasma_B-IGHA1 (genes retained: 3708)
处理完成 Plasma_B-IGHG1 (genes retained: 3620)
处理完成 Plasmablast-MKI67 (genes retained: 2815)
处理完成 cMono-CD14 (genes retained: 10468)
处理完成 ncMono-FCGR3A (genes retained: 9314)
处理完成 ncMono-C1QA (genes retained: 5147)
处理完成 cDC2-CD1C (genes retained: 8617)
处理完成 cMono-IFI44L (genes retained: 7097)
处理完成 ncMono-IFIT1 (genes retained: 5400)
处理完成 pDC-AXL (genes retained: 3235)
处理完成 ncMono-IFI44L (genes retained: 7039)
处理完成 pDC

In [14]:
del adata

#### 过滤掉低于80个样本的细胞类型

In [15]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/01.pseudobulk/'

# 阈值设置
min_sample_threshold = 80

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept ncMono-C1QA.csv — 119 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 141 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Naive_T-SOX4.csv — 137 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 142 samples
[✓] Kept CD4_Tcm-GPR183.csv — 142 samples
[✓] Kept CD4_Tcm-IFIT3.csv — 141 samples
[✓] Kept CD4_Tcm-SOX4.csv — 138 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 140 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 141 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 139 samples
[✓] Kept CD4_Th1-GZMK.csv — 140 samples
[✓] Kept CD4_Th17-RORC.csv — 141 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 141 samples
[✓] Kept CD4_Treg-FOXP3.csv — 142 samples
[✓] Kept ncMono-FCGR3A.csv — 142 samples
[✓] Kept ncMono-IFI44L.csv — 141 samples
[✓] Kept ncMono-IFIT1.csv — 115 samples
[✓] Kept NKT-IFIT3.csv — 88 samples
[✓] Kept NKT-NCR1.csv — 142 samples
[✓] Kept NK_bright-XCL1.csv — 142 samples
[🗑️] Removed pDC-AXL.csv — only 7 samples

In [16]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/02.normal_dis/'

# 阈值设置
min_sample_threshold = 80

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept ncMono-C1QA.csv — 119 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 141 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Naive_T-SOX4.csv — 137 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 142 samples
[✓] Kept CD4_Tcm-GPR183.csv — 142 samples
[✓] Kept CD4_Tcm-IFIT3.csv — 141 samples
[✓] Kept CD4_Tcm-SOX4.csv — 138 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 140 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 141 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 139 samples
[✓] Kept CD4_Th1-GZMK.csv — 140 samples
[✓] Kept CD4_Th17-RORC.csv — 141 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 141 samples
[✓] Kept CD4_Treg-FOXP3.csv — 142 samples
[✓] Kept ncMono-FCGR3A.csv — 142 samples
[✓] Kept ncMono-IFI44L.csv — 141 samples
[✓] Kept ncMono-IFIT1.csv — 115 samples
[✓] Kept NKT-IFIT3.csv — 88 samples
[✓] Kept NKT-NCR1.csv — 142 samples
[✓] Kept NK_bright-XCL1.csv — 142 samples
[🗑️] Removed pDC-AXL.csv — only 7 samples

In [17]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/03.top2000_pseudobulk/'

# 阈值设置
min_sample_threshold = 80

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept CD8_CTL-GZMK.csv — 141 samples
[✓] Kept ncMono-C1QA.csv — 119 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 141 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Naive_T-SOX4.csv — 137 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 142 samples
[✓] Kept CD4_Tcm-GPR183.csv — 142 samples
[✓] Kept CD4_Tcm-IFIT3.csv — 141 samples
[✓] Kept CD4_Tcm-SOX4.csv — 138 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 140 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 141 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 139 samples
[✓] Kept CD4_Th1-GZMK.csv — 140 samples
[✓] Kept CD4_Th17-RORC.csv — 141 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 141 samples
[✓] Kept CD4_Treg-FOXP3.csv — 142 samples
[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept ncMono-FCGR3A.csv — 142 samples
[✓] Kept ncMono-IFI44L.csv — 141 samples
[✓] Kept ncMono-IFIT1.csv — 115 samples
[✓] Kept NKT-IFIT3.csv — 88 samples
[✓] Kept NKT-NCR1.csv — 142 samples
[✓] Kept NK_bright-XCL1.csv — 142 samples
[

In [18]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/04.top2000_normal_dis/'

# 阈值设置
min_sample_threshold = 80

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept CD8_CTL-GZMK.csv — 141 samples
[✓] Kept ncMono-C1QA.csv — 119 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 141 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Naive_T-SOX4.csv — 137 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 142 samples
[✓] Kept CD4_Tcm-GPR183.csv — 142 samples
[✓] Kept CD4_Tcm-IFIT3.csv — 141 samples
[✓] Kept CD4_Tcm-SOX4.csv — 138 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 140 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 141 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 139 samples
[✓] Kept CD4_Th1-GZMK.csv — 140 samples
[✓] Kept CD4_Th17-RORC.csv — 141 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 141 samples
[✓] Kept CD4_Treg-FOXP3.csv — 142 samples
[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept ncMono-FCGR3A.csv — 142 samples
[✓] Kept ncMono-IFI44L.csv — 141 samples
[✓] Kept ncMono-IFIT1.csv — 115 samples
[✓] Kept NKT-IFIT3.csv — 88 samples
[✓] Kept NKT-NCR1.csv — 142 samples
[✓] Kept NK_bright-XCL1.csv — 142 samples
[

#### 种群结构分析（PCA）

In [21]:
import pandas as pd

In [16]:
os.chdir("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/PCA")

In [17]:
#修改格式
!/home/yangshichen/mambaforge/envs/QTL/bin/plink \
--vcf /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/samplePHD_snp_qc_id_chr.vcf.gz \
--make-bed \
--out HIV_snp_qc_id_chr

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to HIV_snp_qc_id_chr.log.
Options in effect:
  --make-bed
  --out HIV_snp_qc_id_chr
  --vcf /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/samplePHD_snp_qc_id_chr.vcf.gz

1546778 MB RAM detected; reserving 773389 MB for main workspace.
--vcf: HIV_snp_qc_id_chr-temporary.bed + HIV_snp_qc_id_chr-temporary.bim +
HIV_snp_qc_id_chr-temporary.fam written.
7540937 variants loaded from .bim file.
142 people (0 males, 0 females, 142 ambiguous) loaded from .fam.
Ambiguous sex IDs written to HIV_snp_qc_id_chr.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 142 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is 0.999915.
7540937 variants and 142 people pass filters and QC.
Note: No phenotypes present.
--make-bed to HIV_snp_qc_i

In [20]:
#PCA
!/home/yangshichen/mambaforge/envs/QTL/bin/plink \
--bfile HIV_snp_qc_id_chr \
--pca 50 \
--out HIV_pca

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to HIV_pca.log.
Options in effect:
  --bfile HIV_snp_qc_id_chr
  --out HIV_pca
  --pca 50

1546778 MB RAM detected; reserving 773389 MB for main workspace.
7540937 variants loaded from .bim file.
142 people (0 males, 0 females, 142 ambiguous) loaded from .fam.
Ambiguous sex IDs written to HIV_pca.nosex .
Using up to 95 threads (change this with --threads).
Before main variant filters, 142 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is 0.999915.
7540937 variants and 142 people pass filters and QC.
Note: No phenotypes present.
Relationship matrix calculation complete.
--pca: Results saved to HIV_pca.eigenval and HIV_pca.eigenvec .


In [29]:
# 读取 eigenvec 文件
eigenvec_df = pd.read_csv("HIV_pca.eigenvec", delim_whitespace=True, header=None)

# 自动生成列名
n_pcs = eigenvec_df.shape[1] - 2
eigenvec_df.columns = ["FID", "IID"] + [f"PC{i+1}" for i in range(n_pcs)]
eigenvec_df.to_csv("eigenvec.csv", index=False)
eigenvec_df

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50
0,HD-H134,HD-H134,-0.018109,0.072116,0.041716,-0.037380,-0.078182,-0.034938,-0.005858,0.052863,...,0.005936,0.075399,0.047025,0.104744,0.046955,-0.047869,-0.019502,-0.088924,0.019070,0.019553
1,HD-H135,HD-H135,-0.056047,0.046050,-0.015293,0.047043,0.025721,-0.091993,0.093621,-0.039609,...,-0.003998,0.109799,0.065985,0.022917,-0.021322,-0.086469,0.095127,0.075873,-0.039925,0.112946
2,HD-H136,HD-H136,-0.037608,0.021948,0.059123,0.081928,-0.056706,-0.100563,-0.044823,0.026030,...,0.017372,0.087057,-0.061628,-0.014723,-0.038816,-0.008551,0.090726,-0.119655,0.061382,-0.062431
3,HD-H137,HD-H137,-0.022011,0.064672,-0.015247,-0.116848,-0.030101,-0.087318,0.117293,0.018941,...,0.035168,0.032029,-0.034428,0.057906,0.003339,0.166563,0.001108,-0.117644,0.004207,0.027102
4,HD-H138,HD-H138,0.025048,0.002615,-0.033241,-0.035601,0.093696,-0.034678,-0.043783,0.082275,...,-0.030157,-0.027228,0.007599,0.037338,-0.076693,-0.185536,0.034063,-0.024598,-0.116223,-0.034846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,PD-H302,PD-H302,0.011409,0.009498,0.059507,0.083481,-0.083159,-0.002140,-0.039550,-0.083234,...,0.041153,-0.059832,-0.120951,0.002643,-0.121515,-0.001460,0.102893,-0.045878,-0.027438,0.084982
138,PD-H305,PD-H305,-0.043431,-0.000091,0.013843,-0.089508,-0.076548,0.036772,0.034883,0.062330,...,-0.056417,0.047383,-0.034629,0.045360,-0.022585,-0.019468,0.003361,-0.038982,0.024271,0.003423
139,PD-H306,PD-H306,-0.023412,0.063077,0.010625,-0.006839,0.081454,0.006327,-0.171125,0.001061,...,0.033614,-0.081586,0.029276,0.139945,0.050840,0.040910,0.150516,-0.044171,0.094908,-0.177374
140,PD-H307,PD-H307,0.009598,0.008123,0.069164,-0.049046,-0.126911,0.127106,-0.039144,0.136938,...,-0.225225,0.063507,0.043040,0.056613,0.181352,0.067107,-0.119742,-0.097232,0.009327,-0.011128


In [30]:
eigenval_df = pd.read_csv("HIV_pca.eigenval", header=None, names=["eigenvalue"])
eigenval_df.to_csv("eigenval.csv", index=False)
eigenval_df

,eigenvalue
0,1.17767
1,1.13380
2,1.11341
3,1.10254
4,1.10152
5,1.09734
6,1.09531
7,1.09257
8,1.08678
9,1.08342


#### PEER（识别基因表达数据中潜在的隐藏协变量）

In [19]:
'''
参见代码:02.tensorQTL-data_PEER.ipynb
'''

'\n参见代码:02.tensorQTL-data_PEER.ipynb\n'

#### 每个基因的残差表达

In [20]:
'''
参见代码:02.tensorQTL-data_PEER.ipynb
'''

'\n参见代码:02.tensorQTL-data_PEER.ipynb\n'

### scATAC-seq数据处理

#### 构建伪bulk数据

In [4]:
adata=sc.read("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/scATAC-peak.h5ad")
adata

AnnData object with n_obs × n_vars = 1226658 × 354883
    obs: 'celltype_L3', 'sample', 'celltype_L1_5', 'people', 'stage', 'celltypesL2_frL3', 'celltypesL1_frL3'

In [6]:
#去除PD-H273样本（与scRNA-seq保持一致）
subadata = adata[adata.obs['people'] != 'PD-H273'].copy()

In [7]:
# 去除（X、Y、MT）染色体上的基因
mask = ~subadata.var_names.str.startswith(('chrX', 'chrY', 'chrMT'))
subadata = subadata[:, mask]
subadata

View of AnnData object with n_obs × n_vars = 1216704 × 343510
    obs: 'celltype_L3', 'sample', 'celltype_L1_5', 'people', 'stage', 'celltypesL2_frL3', 'celltypesL1_frL3'

In [9]:
# 获取所有 celltype_L3 的唯一值
celltypes = subadata.obs['celltype_L3'].unique()
celltypes

['cMono-IFI44L', 'cMono-CD14', 'cDC2-CD1C', 'ncMono-FCGR3A', 'gdT2-GZMH', ..., 'Naive_B-IFIT3', 'ncMono-C1QA', 'CD4_Naive_T-SOX4', 'cMono-IL1B', 'cDC1-BATF3']
Length: 55
Categories (55, object): ['Aptypical_Memory_B-ITGAX', 'CD4_Naive_T-CCR7', 'CD4_Naive_T-SOX4', 'CD4_Tcm-CXCR5', ..., 'ncMono-IFI44L', 'ncMono-IFIT1', 'pDC-AXL', 'pDC-LILRA4']

In [10]:
for celltype in celltypes:
    # 提取该细胞类型的子集
    adata_sub = subadata[subadata.obs['celltype_L3'] == celltype].copy()

    #算伪Bulk
    pdata = dc.get_pseudobulk(
            adata_sub,
            sample_col='people',
            groups_col=None,
            mode='mean',
            min_cells=10,
            min_counts=0,
            min_prop=0,
            min_smpls=0)
        
    #生成伪bulk矩阵
    pseudo_matrix = pd.DataFrame(pdata.X)
    pseudo_matrix.columns = pdata.var_names
    pseudo_matrix.index = pdata.obs.index

    #只选取在90%的样本中都表达的特征
    non_zero_ratio = (pseudo_matrix != 0).mean()
    columns_to_keep = non_zero_ratio[non_zero_ratio >= 0.9].index
    pseudo_matrix = pseudo_matrix[columns_to_keep]

    # 计算每列的均值和标准差
    means = pseudo_matrix.mean()
    stds = pseudo_matrix.std()
    # 计算变异系数
    cv = (stds / means).abs() * 100
    # 按变异系数从大到小排序
    sorted_columns = cv.sort_values(ascending=False).index
    # 选取变异系数最高的前2000列
    top_2000_columns = sorted_columns[:min(2000,pseudo_matrix.shape[1])]

    #保存数据
    if pseudo_matrix.shape[1] > 0:
        normalized_pseudo_matrix = quantile_transformer.fit_transform(pseudo_matrix)
        normalized_pseudo_matrix = scaler.fit_transform(normalized_pseudo_matrix)
        normalized_pseudo_matrix = pd.DataFrame(normalized_pseudo_matrix, columns=pseudo_matrix.columns,index=pseudo_matrix.index)
        normalized_pseudo_matrix.to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/02.normal_dis/{celltype}.csv')
        normalized_pseudo_matrix[top_2000_columns].to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/04.top2000_normal_dis/{celltype}.csv')
        pseudo_matrix.to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/01.pseudobulk/{celltype}.csv')
        pseudo_matrix[top_2000_columns].to_csv(f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/03.top2000_pseudobulk/{celltype}.csv')
    print(f"处理完成 {celltype} (peaks retained: {len(columns_to_keep)})", flush=True)

处理完成 cMono-IFI44L (peaks retained: 60701)
处理完成 cMono-CD14 (peaks retained: 147380)
处理完成 cDC2-CD1C (peaks retained: 51570)
处理完成 ncMono-FCGR3A (peaks retained: 68212)
处理完成 gdT2-GZMH (peaks retained: 15813)
处理完成 CD4_Naive_T-CCR7 (peaks retained: 48194)
处理完成 pDC-LILRA4 (peaks retained: 18357)
处理完成 CD8_CTL-GZMB (peaks retained: 60858)
处理完成 CD4_Tem-CCR7neg (peaks retained: 13332)
处理完成 Unswitched_Memory_B-CD1C (peaks retained: 37825)
处理完成 CD8_Tem-GZMK (peaks retained: 41147)
处理完成 Mature_NK_dim-FCGR3A (peaks retained: 43261)
处理完成 CD8_Naive_T-CCR7 (peaks retained: 43580)
处理完成 CD4_Th17-RORC (peaks retained: 18565)
处理完成 Switched_Memory_B-CD27 (peaks retained: 21062)
处理完成 NKT-NCR1 (peaks retained: 60553)
处理完成 CD4_Treg-FOXP3 (peaks retained: 17178)
处理完成 CD4_Th22-CCR10 (peaks retained: 22830)
处理完成 pDC-AXL (peaks retained: 3852)
处理完成 CD8_Tcm-GZMK (peaks retained: 39755)
处理完成 Aptypical_Memory_B-ITGAX (peaks retained: 15574)
处理完成 Naive_B-TCL1A (peaks retained: 57461)
处理完成 Cycling_T-MKI67 (peaks retaine

#### 过滤掉低于40个样本的细胞类型

In [11]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/01.pseudobulk/'

# 阈值设置
min_sample_threshold = 40

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept cDC2-CD1C.csv — 142 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 98 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 138 samples
[✓] Kept CD4_Tcm-GPR183.csv — 122 samples
[🗑️] Removed CD4_Tcm-IFIT3.csv — only 29 samples
[🗑️] Removed CD4_Tcm-SOX4.csv — only 35 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 99 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 140 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 71 samples
[✓] Kept CD4_Th1-GZMK.csv — 60 samples
[✓] Kept CD4_Th17-RORC.csv — 137 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 127 samples
[✓] Kept CD4_Treg-FOXP3.csv — 139 samples
[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept CD8_CTL-GZMK.csv — 59 samples
[✓] Kept CD8_Naive_T-CCR7.csv — 142 samples
[🗑️] Removed CD8_Tcm-GPR183.csv — only 14 samples
[✓] Kept CD8_Tcm-GZMK.csv — 142 samples
[🗑️] Removed CD8_Tcm-IFI44L.csv — only 7 samples
[✓] Kept CD8_Tem-GZMK.csv — 142 samples
[✓] Kept cMono-CD14.csv — 142 samples
[✓] Kept cM

In [12]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/02.normal_dis/'

# 阈值设置
min_sample_threshold = 40

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept cDC2-CD1C.csv — 142 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 98 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 138 samples
[✓] Kept CD4_Tcm-GPR183.csv — 122 samples
[🗑️] Removed CD4_Tcm-IFIT3.csv — only 29 samples
[🗑️] Removed CD4_Tcm-SOX4.csv — only 35 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 99 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 140 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 71 samples
[✓] Kept CD4_Th1-GZMK.csv — 60 samples
[✓] Kept CD4_Th17-RORC.csv — 137 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 127 samples
[✓] Kept CD4_Treg-FOXP3.csv — 139 samples
[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept CD8_CTL-GZMK.csv — 59 samples
[✓] Kept CD8_Naive_T-CCR7.csv — 142 samples
[🗑️] Removed CD8_Tcm-GPR183.csv — only 14 samples
[✓] Kept CD8_Tcm-GZMK.csv — 142 samples
[🗑️] Removed CD8_Tcm-IFI44L.csv — only 7 samples
[✓] Kept CD8_Tem-GZMK.csv — 142 samples
[✓] Kept cMono-CD14.csv — 142 samples
[✓] Kept cM

In [13]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/03.top2000_pseudobulk/'

# 阈值设置
min_sample_threshold = 40

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept cDC2-CD1C.csv — 142 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 98 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 138 samples
[✓] Kept CD4_Tcm-GPR183.csv — 122 samples
[🗑️] Removed CD4_Tcm-IFIT3.csv — only 29 samples
[🗑️] Removed CD4_Tcm-SOX4.csv — only 35 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 99 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 140 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 71 samples
[✓] Kept CD4_Th1-GZMK.csv — 60 samples
[✓] Kept CD4_Th17-RORC.csv — 137 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 127 samples
[✓] Kept CD4_Treg-FOXP3.csv — 139 samples
[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept CD8_CTL-GZMK.csv — 59 samples
[✓] Kept CD8_Naive_T-CCR7.csv — 142 samples
[🗑️] Removed CD8_Tcm-GPR183.csv — only 14 samples
[✓] Kept CD8_Tcm-GZMK.csv — 142 samples
[🗑️] Removed CD8_Tcm-IFI44L.csv — only 7 samples
[✓] Kept CD8_Tem-GZMK.csv — 142 samples
[✓] Kept cMono-CD14.csv — 142 samples
[✓] Kept cM

In [14]:
# 设置目录路径
input_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/04.top2000_normal_dis/'

# 阈值设置
min_sample_threshold = 40

# 扫描所有 csv 文件
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 统计样本数，并删除样本数过少的文件
for file in csv_files:
    filepath = os.path.join(input_dir, file)
    df = pd.read_csv(filepath, index_col=0)
    
    sample_count = df.shape[0]
    
    if sample_count < min_sample_threshold:
        os.remove(filepath)
        print(f"[🗑️] Removed {file} — only {sample_count} samples")
    else:
        print(f"[✓] Kept {file} — {sample_count} samples")

[✓] Kept cDC2-CD1C.csv — 142 samples
[✓] Kept Aptypical_Memory_B-ITGAX.csv — 98 samples
[✓] Kept CD4_Naive_T-CCR7.csv — 142 samples
[✓] Kept CD4_Tcm-CXCR5.csv — 138 samples
[✓] Kept CD4_Tcm-GPR183.csv — 122 samples
[🗑️] Removed CD4_Tcm-IFIT3.csv — only 29 samples
[🗑️] Removed CD4_Tcm-SOX4.csv — only 35 samples
[✓] Kept CD4_Tem-CCR7neg.csv — 99 samples
[✓] Kept CD4_Tfh_like-CXCR5.csv — 140 samples
[✓] Kept CD4_Th-TNFRSF11A.csv — 71 samples
[✓] Kept CD4_Th1-GZMK.csv — 60 samples
[✓] Kept CD4_Th17-RORC.csv — 137 samples
[✓] Kept CD4_Th22-CCR10.csv — 142 samples
[✓] Kept CD4_Treg-FCRL3.csv — 127 samples
[✓] Kept CD4_Treg-FOXP3.csv — 139 samples
[✓] Kept CD8_CTL-GZMB.csv — 142 samples
[✓] Kept CD8_CTL-GZMK.csv — 59 samples
[✓] Kept CD8_Naive_T-CCR7.csv — 142 samples
[🗑️] Removed CD8_Tcm-GPR183.csv — only 14 samples
[✓] Kept CD8_Tcm-GZMK.csv — 142 samples
[🗑️] Removed CD8_Tcm-IFI44L.csv — only 7 samples
[✓] Kept CD8_Tem-GZMK.csv — 142 samples
[✓] Kept cMono-CD14.csv — 142 samples
[✓] Kept cM

#### PEER（识别peak开放数据中潜在的隐藏协变量）

In [7]:
'''
参见代码:02.tensorQTL-data_PEER.ipynb
'''

'\n参见代码:02.tensorQTL-data_PEER.ipynb\n'

#### 每个peak的残差开放

In [8]:
'''
参见代码:02.tensorQTL-data_PEER.ipynb
'''

'\n参见代码:02.tensorQTL-data_PEER.ipynb\n'

### WGS数据

#### 加载WGS数据

In [4]:
import allel
import pandas as pd

In [5]:
# 读取 VCF 文件（支持 .vcf.gz）
callset = allel.read_vcf("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/samplePHD_snp_qc_id_chr.vcf.gz",
                        fields=["samples", "variants/CHROM", "variants/POS", "variants/ID", "variants/REF", "variants/ALT", "calldata/GT"])

In [6]:
callset

{'samples': array(['HD-H134_HD-H134', 'HD-H135_HD-H135', 'HD-H136_HD-H136',
        'HD-H137_HD-H137', 'HD-H138_HD-H138', 'HD-H140_HD-H140',
        'HD-H141_HD-H141', 'HD-H143_HD-H143', 'HD-H145_HD-H145',
        'HD-H146_HD-H146', 'HD-H147_HD-H147', 'HD-H148_HD-H148',
        'HD-H150_HD-H150', 'HD-H153_HD-H153', 'HD-H154_HD-H154',
        'HD-H155_HD-H155', 'HD-H156_HD-H156', 'HD-H157_HD-H157',
        'HD-H160_HD-H160', 'HD-H161_HD-H161', 'HD-H162_HD-H162',
        'HD-H273_HD-H273', 'HD-H274_HD-H274', 'HD-H275_HD-H275',
        'HD-H276_HD-H276', 'HD-H277_HD-H277', 'HD-H279_HD-H279',
        'HD-H280-1_HD-H280-1', 'HD-H280-2_HD-H280-2', 'HD-H309_HD-H309',
        'HD-H311_HD-H311', 'HD-H312_HD-H312', 'HD-H313_HD-H313',
        'HD-H314_HD-H314', 'HD-H315_HD-H315', 'HD-H316_HD-H316',
        'HD-H317_HD-H317', 'HD-H318_HD-H318', 'HD-H319_HD-H319',
        'HD-H320_HD-H320', 'HD-H321_HD-H321', 'HD-H322_HD-H322',
        'HD-H323_HD-H323', 'HD-H324_HD-H324', 'HD-H325_HD-H325',
      

In [7]:
# 提取样本名
samples = callset["samples"]
len(samples)

142

#### 构建vatiant_df（每个变异位点的注释信息）

In [30]:
variant_df = pd.DataFrame({
    "variant_id": callset["variants/ID"],
    "chrom": callset["variants/CHROM"],
    "pos": callset["variants/POS"],
    "ref": callset["variants/REF"],
    "alt": [a[0] if len(a) > 0 else None for a in callset["variants/ALT"]]
})
variant_df = variant_df.dropna(subset=["variant_id"])  # 过滤掉 ID 缺失的 SNP
variant_df.set_index('variant_id', inplace=True)
variant_df

,chrom,pos,ref,alt
variant_id,,,,
1_16487_T_[C],chr1,16487,T,C
1_16534_C_[T],chr1,16534,C,T
1_17746_A_[G],chr1,17746,A,G
1_49298_C_[T],chr1,49298,C,T
1_51206_G_[A],chr1,51206,G,A
...,...,...,...,...
22_50801268_T_[C],chr22,50801268,T,C
22_50806741_C_[T],chr22,50806741,C,T
22_50807539_C_[G],chr22,50807539,C,G


In [31]:
variant_df.to_parquet("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/142_sample_variant_df.parquet")

#### 构建genotype_df（每个样本的基因型矩阵，转换为 0/1/2 编码）

In [11]:
gt = allel.GenotypeArray(callset["calldata/GT"]).to_n_alt()
genotype_df = pd.DataFrame(gt, columns=samples, index=variant_df["variant_id"])
genotype_df.columns = [col.split('_')[0] for col in genotype_df.columns]

In [12]:
genotype_df

,HD-H134,HD-H135,HD-H136,HD-H137,HD-H138,HD-H140,HD-H141,HD-H143,HD-H145,HD-H146,...,PD-H296,PD-H297,PD-H299,PD-H300,PD-H301,PD-H302,PD-H305,PD-H306,PD-H307,PD-H308
variant_id,,,,,,,,,,,,,,,,,,,,,
1_16487_T_[C],0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1_16534_C_[T],0,1,1,0,0,1,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1_17746_A_[G],1,0,1,0,0,0,1,0,0,1,...,0,0,0,0,1,0,1,0,1,1
1_49298_C_[T],0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1
1_51206_G_[A],0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_50801268_T_[C],0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
22_50806741_C_[T],0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
22_50807539_C_[G],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
genotype_df.to_parquet("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/142_sample_genotype.parquet")

### Use tensorqtl-eQTL

#### Gene expression.csv转bed

In [4]:
# 输入目录、输出目录
input_dir = "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/02.normal_dis/"
output_dir1 = "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/06.gene_expression_bed/"
output_dir2 = "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scRNA-seq/07.filter_normal_dis//"

# 基因注释表（必须包含 gene_id, chrom, start, end）
annotation_df = pd.read_csv("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/gene_annotation.txt", sep='\t')
annotation_df = annotation_df.rename(columns={
    "gene_id": "gene_id",
    "chr": "chr",
    "left": "start",
    "right": "end"})
annotation_df

,gene_id,chr,start,end
0,DDX11L17,chr1,182696,184174
1,PRDM16,chr1,3069168,3438621
2,PEX10,chr1,2403964,2413797
3,RPL21P21,chr1,10054445,10054781
4,LINC01345,chr1,3944547,3949024
...,...,...,...,...
41008,ZNF736P9Y,chrY,8068896,8070723
41009,FAM197Y8,chrY,9354846,9356814
41010,PRKY,chrY,7274313,7371868
41011,SEPTIN14P23,chrY,25378300,25394719


In [5]:
import pandas as pd
import re
def chrom_sort_key(chrom):
    # 提取数字部分（chr1 → 1, chrX → 23, chrY → 24）
    match = re.match(r'chr(\d+)$', chrom)
    if match:
        return int(match.group(1))
    elif chrom == 'chrX':
        return 23
    elif chrom == 'chrY':
        return 24
    elif chrom == 'chrM':
        return 25
    else:
        return 100  # 把奇怪的染色体排到后面

In [6]:
#遍历所有csv文件
for fname in os.listdir(input_dir):
    if fname.endswith(".csv"):
        print("Processing:", fname)
        expr_df = pd.read_csv(os.path.join(input_dir, fname), index_col=0).T
        expr_df.index.name = 'gene_id'
        expr_df.reset_index(inplace=True)

        #合并注释
        merged_df = annotation_df.merge(expr_df, on='gene_id')
        merged_df = merged_df[['chr', 'start', 'end', 'gene_id'] + list(expr_df.columns[1:])]

        #按照染色体顺序排序（否则会报错）
        autosomes = [f"chr{i}" for i in range(1, 23)]
        merged_df = merged_df[merged_df["chr"].isin(autosomes)]
        merged_df['chr_sort_key'] = merged_df['chr'].apply(chrom_sort_key)
        merged_df = merged_df.sort_values(by=['chr_sort_key', 'start', 'end']).drop(columns='chr_sort_key').reset_index(drop=True)

        #保存为bed和csv格式
        bed_fname = fname.replace(".csv", ".bed")
        merged_df.to_csv(os.path.join(output_dir1, bed_fname), sep='\t', index=False)
        merged_df.to_csv(os.path.join(output_dir2, fname), index=False)
        print("Processing:", fname, "Done!")

Processing: CD8_CTL-GZMB.csv
Processing: CD8_CTL-GZMB.csv Done!
Processing: ncMono-C1QA.csv
Processing: ncMono-C1QA.csv Done!
Processing: Aptypical_Memory_B-ITGAX.csv
Processing: Aptypical_Memory_B-ITGAX.csv Done!
Processing: CD4_Naive_T-CCR7.csv
Processing: CD4_Naive_T-CCR7.csv Done!
Processing: CD4_Naive_T-SOX4.csv
Processing: CD4_Naive_T-SOX4.csv Done!
Processing: CD4_Tcm-CXCR5.csv
Processing: CD4_Tcm-CXCR5.csv Done!
Processing: CD4_Tcm-GPR183.csv
Processing: CD4_Tcm-GPR183.csv Done!
Processing: CD4_Tcm-IFIT3.csv
Processing: CD4_Tcm-IFIT3.csv Done!
Processing: CD4_Tcm-SOX4.csv
Processing: CD4_Tcm-SOX4.csv Done!
Processing: CD4_Tem-CCR7neg.csv
Processing: CD4_Tem-CCR7neg.csv Done!
Processing: CD4_Tfh_like-CXCR5.csv
Processing: CD4_Tfh_like-CXCR5.csv Done!
Processing: CD4_Th-TNFRSF11A.csv
Processing: CD4_Th-TNFRSF11A.csv Done!
Processing: CD4_Th1-GZMK.csv
Processing: CD4_Th1-GZMK.csv Done!
Processing: CD4_Th17-RORC.csv
Processing: CD4_Th17-RORC.csv Done!
Processing: CD4_Th22-CCR10.csv

In [7]:
merged_df

,chr,start,end,gene_id,HD-H134,HD-H135,HD-H136,HD-H137,HD-H138,HD-H140,...,PD-H296,PD-H297,PD-H299,PD-H300,PD-H301,PD-H302,PD-H305,PD-H306,PD-H307,PD-H308
0,chr1,868071,876903,FAM41C,-0.565845,0.029931,1.361445,-0.021413,0.789345,0.708622,...,0.880448,0.488615,0.081112,0.653065,-0.893189,-0.047229,0.400176,-0.924981,-3.679358,0.551286
1,chr1,944203,959309,NOC2L,-0.244069,-0.180023,-0.634448,-0.164174,-0.500075,-0.481876,...,0.614388,0.883923,0.393747,1.199229,-1.579108,-2.144466,0.518493,-1.665978,-0.556043,0.342783
2,chr1,960584,965719,KLHL17,0.010913,-0.402288,-0.144625,0.266456,0.024923,0.281139,...,-0.059322,0.066955,-0.159018,-0.452089,-1.266761,-1.469270,4.128454,-0.370038,-0.435283,-0.741560
3,chr1,1001138,1014540,ISG15,-0.518496,-0.740342,-0.446088,-0.309454,0.116935,0.428481,...,0.518504,0.101269,0.244076,-0.101261,-4.546250,-2.144492,1.089492,0.393757,-0.809406,0.654860
4,chr1,1020120,1056118,AGRN,0.373717,-0.733924,-0.686969,-1.017667,-0.393885,-0.028750,...,-0.491627,0.045349,0.156101,0.678941,-0.304761,-3.516830,0.661166,0.069929,0.180893,0.414696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10339,chr22,50568861,50578465,CPT1B,-0.910235,-0.359634,-1.773302,-0.937397,0.809399,-0.556043,...,-0.132633,0.376619,-0.481877,0.614389,-4.546193,-0.833568,0.858390,-0.575205,-0.411028,0.937397
10340,chr22,50578959,50601455,CHKB,0.411028,-0.148380,0.180023,0.762827,0.359634,1.056470,...,-0.342783,0.614388,0.910235,0.070024,-2.144466,-4.546191,1.240356,-1.505503,-1.916768,0.054440
10341,chr22,50583026,50595634,CHKB-DT,0.853066,-0.447224,-0.589177,-0.983659,-0.336229,0.664743,...,0.461483,0.067290,1.113501,0.807098,-1.056898,-0.950448,-0.321086,0.356399,-1.142440,0.492961
10342,chr22,50622754,50628173,ARSA,0.132632,0.276568,-0.463889,-0.446091,0.654849,-0.883931,...,0.965499,0.762830,-0.556048,-1.072789,-1.332036,-4.546222,1.916778,-1.665990,-0.809406,-0.195934


#### Read genotype_df and vatiant_df

In [6]:
genotype_df = pd.read_parquet('/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/01.Genotype/142_sample_genotype.parquet')
variant_df = pd.read_parquet('/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/01.Genotype/142_sample_variant_df.parquet')

#### Run tensorqtl

In [11]:
#设置bed文件所在的文件夹路径
bed_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/02.scRNA-seq/01.pseudobulk/06.gene_expression_bed/'
output_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/03.Result/eQTL2/'

#获取所有以.bed结尾的文件
bed_files = [f for f in os.listdir(bed_dir) if f.endswith('.bed')]

In [12]:
bed_files

['CD8_Naive_T-CCR7.bed',
 'ncMono-FCGR3A.bed',
 'Aptypical_Memory_B-ITGAX.bed',
 'CD4_Naive_T-CCR7.bed',
 'CD4_Naive_T-SOX4.bed',
 'CD4_Tcm-CXCR5.bed',
 'CD4_Tcm-GPR183.bed',
 'CD4_Tcm-IFIT3.bed',
 'CD4_Tcm-SOX4.bed',
 'CD4_Tem-CCR7neg.bed',
 'CD4_Tfh_like-CXCR5.bed',
 'CD4_Th-TNFRSF11A.bed',
 'CD4_Th1-GZMK.bed',
 'CD4_Th17-RORC.bed',
 'CD4_Th22-CCR10.bed',
 'CD4_Treg-FCRL3.bed',
 'CD4_Treg-FOXP3.bed',
 'CD8_CTL-GZMB.bed',
 'CD8_CTL-GZMK.bed',
 'ncMono-IFI44L.bed',
 'ncMono-IFIT1.bed',
 'NKT-IFIT3.bed',
 'NKT-NCR1.bed',
 'NK_bright-XCL1.bed',
 'pDC-LILRA4.bed',
 'Plasma_B-IGHA1.bed',
 'Switched_Memory_B-CD27.bed',
 'Switched_Memory_B-CD86.bed',
 'Switched_Memory_B-IGHE.bed',
 'Terminal_NK_dim-CD160neg.bed',
 'Transitional_B-NEIL1.bed',
 'Transitional_NK-GZMK.bed',
 'Unswitched_Memory_B-CD1C.bed',
 'Unswitched_Memory_B-IFIT3.bed',
 'Unswitched_Memory_B-JAM3.bed',
 'CD8_Tcm-GPR183.bed',
 'CD8_Tcm-GZMK.bed',
 'CD8_Tcm-IFI44L.bed',
 'CD8_Tem-GZMK.bed',
 'cDC2-CD1C.bed',
 'cMono-CD14.bed',


In [13]:
# 遍历文件，处理并创建对应文件夹
for bed_file in bed_files:
    cell_name = os.path.splitext(bed_file)[0]  # 去掉.bed后缀
    folder_path = os.path.join(output_dir, cell_name)
    os.makedirs(folder_path, exist_ok=True)  # 创建文件夹

    #read_expression_file and covariates
    expression_bed = f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/02.scRNA-seq/01.pseudobulk/06.gene_expression_bed/{cell_name}.bed'
    covariates_file = f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/02.scRNA-seq/01.pseudobulk/05.PEER/01.factor/{cell_name}.csv'

    #load phenotypes and covariates
    #align phenotypes and covariates
    phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
    covariates_df = pd.read_csv(covariates_file, sep=',', index_col=0)
    phenotype_df = phenotype_df[covariates_df.index]

    #decited the number of peer factor
    sample_size = covariates_df.shape[0]
    if sample_size <= 150:
        PF = list('PF' + str(i) for i in range(1, 16))
    elif sample_size <= 250:
        PF = list('PF' + str(i) for i in range(1, 31))
    elif sample_size > 250:xw
        PF = list('PF' + str(i) for i in range(1, 36))
    info = list(['age','PC1', 'PC2'])
    cova_need = info + PF
    covariates_df = covariates_df.loc[:,cova_need]

    #cis-QTL mapping: permutations (对每个基因，寻找在附近（cis-window，一般是±1Mb）的SNP是否与其表达量显著相关。)
    cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df, 
                         maf_threshold=0.1, seed=8888)

    #后续统一再在py里面算（环境问题）
    #tensorqtl.calculate_qvalues(cis_df)

    #保存
    cis_df.to_csv(f'{folder_path}/all_lead_perm.csv')

    #cis-QTL mapping: summary statistics for all variant-phenotype pairs（所有变异-表型对的统计汇总）
    cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, 'all', covariates_df, maf_threshold=0.1, 
                    output_dir=f'{folder_path}')

cis-QTL mapping: empirical p-values for phenotypes
  * 142 samples
  * 11197 phenotypes
  * 18 covariates
  * 7540937 variants
  * applying in-sample 0.1 MAF filter
  * cis-window: ±1,000,000
  * using seed 8888
  * checking phenotypes: 11197/11197
    ** dropping 9 phenotypes without variants in cis-window
  * computing permutations
    processing phenotype 11188/11188
  Time elapsed: 11.48 min
done.
cis-QTL mapping: nominal associations for all variant-phenotype pairs
  * 142 samples
  * 11197 phenotypes
  * 18 covariates
  * 7540937 variants
  * applying in-sample 0.1 MAF filter
  * cis-window: ±1,000,000
  * checking phenotypes: 11197/11197
    ** dropping 9 phenotypes without variants in cis-window
  * Computing associations
    Mapping chromosome chr1
    processing phenotype 1174/11188    time elapsed: 0.24 min
    * writing output
    Mapping chromosome chr2
    processing phenotype 1939/11188    time elapsed: 0.52 min
    * writing output
    Mapping chromosome chr3
    proces

### Use tensorqtl-caQTL

#### peak accessibility.csv转bed

In [8]:
# 输入目录、输出目录
input_dir = "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/02.normal_dis/"
output_dir1 = "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/06.peak_accessibility_bed/"
output_dir2 = "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/scATAC-seq/07.filter_normal_dis/"

# 基因注释表（必须包含 gene_id, chrom, start, end）
annotation_df = pd.read_csv("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/gene_annotation.txt", sep='\t')
annotation_df = annotation_df.rename(columns={
    "gene_id": "gene_id",
    "chr": "chr",
    "left": "start",
    "right": "end"})
annotation_df

,gene_id,chr,start,end
0,DDX11L17,chr1,182696,184174
1,PRDM16,chr1,3069168,3438621
2,PEX10,chr1,2403964,2413797
3,RPL21P21,chr1,10054445,10054781
4,LINC01345,chr1,3944547,3949024
...,...,...,...,...
41008,ZNF736P9Y,chrY,8068896,8070723
41009,FAM197Y8,chrY,9354846,9356814
41010,PRKY,chrY,7274313,7371868
41011,SEPTIN14P23,chrY,25378300,25394719


In [9]:
import pandas as pd
import re
def chrom_sort_key(chrom):
    # 提取数字部分（chr1 → 1, chrX → 23, chrY → 24）
    match = re.match(r'chr(\d+)$', chrom)
    if match:
        return int(match.group(1))
    elif chrom == 'chrX':
        return 23
    elif chrom == 'chrY':
        return 24
    elif chrom == 'chrM':
        return 25
    else:
        return 100  # 把奇怪的染色体排到后面

In [25]:
import pandas as pd
import pyranges as pr

#遍历所有csv文件
for fname in os.listdir(input_dir):
    if fname.endswith(".csv"):
        print("Processing:", fname)
        expr_df = pd.read_csv(os.path.join(input_dir, fname), index_col=0).T
        expr_df.index.name = 'gene_id'
        expr_df.reset_index(inplace=True)
        expr_df[['Chromosome', 'region']] = expr_df['gene_id'].str.split(':', expand=True)
        expr_df[['Start', 'End']] = expr_df['region'].str.split('-', expand=True).astype(int)
        expr_df.drop(columns='region', inplace=True)
        expr_df['gene_id'] = expr_df['gene_id']

        annotation_df2 = annotation_df
        annotation_df2.columns = ['gene_id', 'Chromosome', 'Start', 'End']

        # 转为 PyRanges 对象
        gr1 = pr.PyRanges(expr_df)
        gr2 = pr.PyRanges(annotation_df)
        
        # 执行 overlap 匹配
        overlaps = gr1.join(gr2)
        
        # overlaps 是一个 PyRanges 对象，转回 pandas 看结果
        matched_df_raw = overlaps.df
        
        # 计算重叠长度
        matched_df_raw["overlap_len"] = (matched_df_raw[["End", "End_b"]].min(axis=1) - matched_df_raw[["Start", "Start_b"]].max(axis=1))
        
        # 对每个 gene_id 选择 overlap 最长的
        matched_df = (matched_df_raw.sort_values("overlap_len", ascending=False)
                      .groupby("gene_id")
                      .first()
                      .reset_index())
        #排列
        cols = matched_df.columns.tolist()
        last6 = cols[-7:]
        others = cols[:-7]
        matched_df = matched_df[last6 + others]

        #修改列名
        merged_df = matched_df.drop(columns=['Start_b', 'End_b', 'gene_id_b', 'overlap_len'])
        merged_df = merged_df.rename(columns={'Chromosome': 'chr',
                                                'Start': 'start',
                                                'End': 'end',})

        #按照染色体顺序排序（否则会报错）
        autosomes = [f"chr{i}" for i in range(1, 23)]
        merged_df = merged_df[merged_df["chr"].isin(autosomes)]
        merged_df['chr_sort_key'] = merged_df['chr'].apply(chrom_sort_key)
        merged_df['chr_sort_key'] = merged_df['chr'].str.replace("chr", "").astype(int)
        merged_df = merged_df.sort_values(by=['chr_sort_key', 'start', 'end']).drop(columns='chr_sort_key').reset_index(drop=True)


        merged_df
        #保存为bed和csv格式
        bed_fname = fname.replace(".csv", ".bed")
        merged_df.to_csv(os.path.join(output_dir1, bed_fname), sep='\t', index=False)
        merged_df.to_csv(os.path.join(output_dir2, fname), index=False)
        print("Processing:", fname, "Done!")

Processing: cDC2-CD1C.csv
Processing: cDC2-CD1C.csv Done!
Processing: Aptypical_Memory_B-ITGAX.csv
Processing: Aptypical_Memory_B-ITGAX.csv Done!
Processing: CD4_Naive_T-CCR7.csv
Processing: CD4_Naive_T-CCR7.csv Done!
Processing: CD4_Tcm-CXCR5.csv
Processing: CD4_Tcm-CXCR5.csv Done!
Processing: CD4_Tcm-GPR183.csv
Processing: CD4_Tcm-GPR183.csv Done!
Processing: CD4_Tem-CCR7neg.csv
Processing: CD4_Tem-CCR7neg.csv Done!
Processing: CD4_Tfh_like-CXCR5.csv
Processing: CD4_Tfh_like-CXCR5.csv Done!
Processing: CD4_Th-TNFRSF11A.csv
Processing: CD4_Th-TNFRSF11A.csv Done!
Processing: CD4_Th1-GZMK.csv
Processing: CD4_Th1-GZMK.csv Done!
Processing: CD4_Th17-RORC.csv
Processing: CD4_Th17-RORC.csv Done!
Processing: CD4_Th22-CCR10.csv
Processing: CD4_Th22-CCR10.csv Done!
Processing: CD4_Treg-FCRL3.csv
Processing: CD4_Treg-FCRL3.csv Done!
Processing: CD4_Treg-FOXP3.csv
Processing: CD4_Treg-FOXP3.csv Done!
Processing: CD8_CTL-GZMB.csv
Processing: CD8_CTL-GZMB.csv Done!
Processing: CD8_CTL-GZMK.csv
Pro

In [26]:
merged_df

,chr,start,end,gene_id,HD-H134,HD-H135,HD-H136,HD-H137,HD-H138,HD-H140,...,PD-H296,PD-H297,PD-H299,PD-H300,PD-H301,PD-H302,PD-H305,PD-H306,PD-H307,PD-H308
0,chr1,778498,778999,chr1:778498-778999,-0.164176,-0.994630,0.309453,-0.211905,-0.085631,1.024909,...,-0.883926,-0.148381,-0.023320,0.809400,-1.383940,-0.195932,0.575205,1.441242,-0.292961,0.211903
1,chr1,827257,827758,chr1:827257-827758,-0.107510,-0.480909,-0.515688,-0.463938,0.067291,0.327540,...,-1.097819,-0.233049,-0.218774,-0.121185,-0.860940,0.877225,0.161262,0.341916,-3.860850,0.257057
2,chr1,869641,870142,chr1:869641-870142,-3.371044,-0.526688,-0.263691,0.603663,-0.210304,0.972824,...,-0.964012,0.036106,0.945048,-0.277386,-0.794011,0.782255,0.802930,1.548404,-3.371044,0.484071
3,chr1,959080,959581,chr1:959080-959581,-0.130293,-0.920055,0.550240,0.066954,0.449929,1.112544,...,-1.469280,0.679203,1.044850,-0.261886,-0.418690,-0.719198,-0.567466,0.761271,0.251855,-0.503909
4,chr1,1000649,1001150,chr1:1000649-1001150,-0.676619,0.196661,0.930980,-0.807874,0.708259,0.856112,...,0.074346,0.055157,0.617548,0.170704,-0.835034,-0.193114,-0.086598,0.832988,-3.679757,-0.547710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29388,chr22,50542399,50542900,chr22:50542399-50542900,-0.860939,-0.570254,-0.589176,-0.190502,-0.463937,0.229409,...,-0.648547,0.188379,0.202003,-0.447223,-0.669322,-0.782733,-0.398471,0.785156,0.461489,0.053929
29389,chr22,50562689,50563190,chr22:50562689-50563190,-0.211903,-1.056472,-0.260273,-0.326059,0.101266,0.342786,...,-1.505505,0.446089,0.195933,-0.038872,-0.675609,1.579114,0.537148,0.821409,-0.276567,0.740339
29390,chr22,50582730,50583231,chr22:50582730-50583231,-1.056472,-0.965497,0.038875,0.244072,1.284433,1.160601,...,-1.089475,-0.164173,0.718328,-0.309453,-1.665982,0.101266,0.342786,-0.276567,-0.718325,0.180025
29391,chr22,50628009,50628510,chr22:50628009-50628510,-0.080469,-0.447423,0.664661,0.107299,0.284871,0.928241,...,-0.770782,-0.162736,0.270862,-0.204768,0.067146,3.955589,0.154372,-0.066931,-0.669544,-0.515893


#### Read genotype_df and vatiant_df

In [4]:
genotype_df = pd.read_parquet('/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/01.Genotype/142_sample_genotype.parquet')
variant_df = pd.read_parquet('/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/01.Genotype/142_sample_variant_df.parquet')

#### Run tensorqtl

In [5]:
#设置bed文件所在的文件夹路径
bed_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/03.scATAC-seq/01.pseudobulk/06.peak_accessibility_bed/'
output_dir = '/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/03.Result/caQTL/'

#获取所有以.bed结尾的文件
bed_files = [f for f in os.listdir(bed_dir) if f.endswith('.bed')]

In [6]:
bed_files2 = bed_files[17:]
bed_files2

['CD8_Tem-GZMK.bed',
 'cDC2-CD1C.bed',
 'cMono-CD14.bed',
 'gdT2-GZMH.bed',
 'MAIT-SLC4A10.bed',
 'Mature_NK_dim-FCGR3A.bed',
 'Naive_B-TCL1A.bed',
 'ncMono-FCGR3A.bed',
 'ncMono-IFI44L.bed',
 'ncMono-IFIT1.bed',
 'NKT-NCR1.bed',
 'NK_bright-XCL1.bed',
 'pDC-LILRA4.bed',
 'Plasma_B-IGHA1.bed',
 'Switched_Memory_B-CD27.bed',
 'Switched_Memory_B-CD86.bed',
 'Switched_Memory_B-IGHE.bed',
 'Unswitched_Memory_B-CD1C.bed']

In [7]:
# 遍历文件，处理并创建对应文件夹
for bed_file in bed_files2:
    cell_name = os.path.splitext(bed_file)[0]  # 去掉.bed后缀
    folder_path = os.path.join(output_dir, cell_name)
    os.makedirs(folder_path, exist_ok=True)  # 创建文件夹

    #read_expression_file and covariates
    expression_bed = f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/03.scATAC-seq/01.pseudobulk/06.peak_accessibility_bed/{cell_name}.bed'
    covariates_file = f'/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/01.Dynamic/01.Data/03.scATAC-seq/01.pseudobulk/05.PEER/01.factor/{cell_name}.csv'

    #load phenotypes and covariates
    #align phenotypes and covariates
    phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
    covariates_df = pd.read_csv(covariates_file, sep=',', index_col=0)
    phenotype_df = phenotype_df[covariates_df.index]

    #decited the number of peer factor
    sample_size = covariates_df.shape[0]
    if sample_size <= 150:
        PF = list('PF' + str(i) for i in range(1, 16))
    elif sample_size <= 250:
        PF = list('PF' + str(i) for i in range(1, 31))
    elif sample_size > 250:
        PF = list('PF' + str(i) for i in range(1, 36))
    info = list(['age', 'treat_time','PC1', 'PC2'])
    cova_need = info + PF
    covariates_df = covariates_df.loc[:,cova_need]

    #cis-QTL mapping: permutations (对每个基因，寻找在附近（cis-window，一般是±1Mb）的SNP是否与其表达量显著相关。)
    cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df, 
                         maf_threshold=0.1, seed=8888)

    #后续统一再在py里面算（环境问题）
    #tensorqtl.calculate_qvalues(cis_df)

    #保存
    cis_df.to_csv(f'{folder_path}/all_lead_perm.csv')

    #cis-QTL mapping: summary statistics for all variant-phenotype pairs（所有变异-表型对的统计汇总）
    cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, 'all', covariates_df, maf_threshold=0.1, 
                    output_dir=f'{folder_path}')

cis-QTL mapping: empirical p-values for phenotypes
  * 142 samples
  * 32614 phenotypes
  * 19 covariates
  * 7540937 variants
  * applying in-sample 0.1 MAF filter
  * cis-window: ±1,000,000
  * using seed 8888
  * checking phenotypes: 32614/32614
    ** dropping 28 phenotypes without variants in cis-window
  * computing permutations
    processing phenotype 32586/32586
  Time elapsed: 62.78 min
done.
cis-QTL mapping: nominal associations for all variant-phenotype pairs
  * 142 samples
  * 32614 phenotypes
  * 19 covariates
  * 7540937 variants
  * applying in-sample 0.1 MAF filter
  * cis-window: ±1,000,000
  * checking phenotypes: 32614/32614
    ** dropping 28 phenotypes without variants in cis-window
  * Computing associations
    Mapping chromosome chr1
    processing phenotype 3395/32586    time elapsed: 0.42 min
    * writing output
    Mapping chromosome chr2
    processing phenotype 5805/32586    time elapsed: 1.25 min
    * writing output
    Mapping chromosome chr3
    proc